In [18]:
# i want to finetune a model to do credit card fraud detection. A sample input is given below.

# the huggingface dataset for legitimate transactions is called "LouisXO/fraud-detection-legitimate"

# the huggingface dataset for fraudulent transactions is called "LouisXO/fraud-detection-all-fraud"

# all datasets have columns "conversation" and "response" 
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR

# response: LEGITIMATE



In [19]:
# Import libraries
import datasets
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

In [20]:
# Load legitimate transactions dataset
legitimate_dataset = load_dataset("LouisXO/fraud-detection-legitimate")

# Load fraudulent transactions dataset
fraudulent_dataset = load_dataset("LouisXO/fraud-detection-all-fraud")

In [21]:
# Assign label 0 to legitimate transactions
legitimate_dataset = legitimate_dataset.map(lambda x: {'label': 0})

# Assign label 1 to fraudulent transactions
fraudulent_dataset = fraudulent_dataset.map(lambda x: {'label': 1})

# Combine the datasets
full_dataset = concatenate_datasets([legitimate_dataset['train'], fraudulent_dataset['train']])

Map: 100%|██████████| 7506/7506 [00:00<00:00, 41555.01 examples/s]


In [22]:
# Shuffle the combined dataset
full_dataset = full_dataset.shuffle(seed=42)

# Split into training and validation sets (e.g., 90% train, 10% validation)
split_dataset = full_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [23]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(example):
    return tokenizer(example['conversation'], padding='max_length', truncation=True)

# Apply the tokenizer to the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 10751/10751 [00:01<00:00, 6117.09 examples/s]


In [24]:
# Load a pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
training_args = TrainingArguments(
    output_dir='./results',            # Output directory
    evaluation_strategy='steps',       # Evaluate every N steps
    save_strategy='steps',             # Save the model every N steps
    eval_steps=500,                    # Evaluation interval
    save_steps=500,                    # Save interval
    num_train_epochs=3,                # Number of training epochs
    per_device_train_batch_size=16,    # Batch size for training
    per_device_eval_batch_size=64,     # Batch size for evaluation
    logging_dir='./logs',              # Directory for logs
    logging_steps=10,
    load_best_model_at_end=True,       # Load the best model when finished training
)

In [27]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

  0%|          | 0/18144 [00:00<?, ?it/s]c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  0%|          | 10/18144 [00:03<1:42:25,  2.95it/s]

{'loss': 0.4914, 'grad_norm': 1.5420668125152588, 'learning_rate': 4.997244268077602e-05, 'epoch': 0.0}


  0%|          | 20/18144 [00:07<1:40:04,  3.02it/s]

{'loss': 0.1912, 'grad_norm': 0.4340767562389374, 'learning_rate': 4.994488536155203e-05, 'epoch': 0.0}


  0%|          | 30/18144 [00:10<1:40:13,  3.01it/s]

...

{'loss': 0.0882, 'grad_norm': 0.40558701753616333, 'learning_rate': 4.86496913580247e-05, 'epoch': 0.08}


  3%|▎         | 500/18144 [02:46<1:37:03,  3.03it/s]

{'loss': 0.0806, 'grad_norm': 0.472676157951355, 'learning_rate': 4.862213403880071e-05, 'epoch': 0.08}



  3%|▎         | 500/18144 [03:55<1:37:03,  3.03it/s]

{'eval_loss': 0.1468181610107422, 'eval_runtime': 68.9891, 'eval_samples_per_second': 155.836, 'eval_steps_per_second': 2.435, 'epoch': 0.08}


In [ ]:
# Evaluate the model
evaluation_results = trainer.evaluate()

# Print evaluation results
print(evaluation_results)